# Analysis Ready Data Tutorial

Time-series analysis (e.g. change detection and trend detection) is a powerful application of satellite imagery. However, a great deal of processing is required to prepare imagery for analysis. Analysis Ready Data (ARD), preprocessed time-series stacks of overhead imagery, allow for time-series analysis without any additional processing of the imagery. See [Analysis Data Defined](https://medium.com/planet-stories/analysis-ready-data-defined-5694f6f48815) for an excellent introduction and discussion on ARD.

This tutorial shows how [Planet APIs](https://developers.planet.com/docs/apis/) can simplify production of ARD by working through two real-world use cases. This tutorial is targeted to users who have little to no geospatial knowledge but have experience working with APIs. The goal of this tutorial is to teach the user the how and whys of using the Data and Orders APIs to create and interpret ARD for both use cases. The use cases addressed in this tutorial are:

* As a software engineer at an ag-tech company, I'd like to be able to order Planet imagery programmatically in a way that enables the data scientist at my organization to create time-series algorithms (e.g. monitoring ndvi curves over time) without further data cleaning and processing.
* As a tropical forest monitoring analyst, I'd like to create an imagery pipeline that delivers analysis-ready imagery over a protected area so I can create simple change detection (ndvi_xdate - ndvi_ydate) analyses. I'd like to do minimal custom (non-Planet) data processing to get the imagery in a format that supports a change-detection analysis because my technical skills are limited.

## APIs

The two Planet APIs that are used in creation of ARD are the [Data API](https://developers.planet.com/docs/data/) and the [Orders API](https://developers.planet.com/docs/orders/). The Data API is used to search for imagery according to certain criteria based on the use case, and the Orders API is used to process that imagery into ARD that can be fed directly into time-series analysis.

### Data API

The first step in using the Data API is identifying the search criteria. This is specifying answers to the following questions regarding the use case time-series analysis:
* What is the time range?
* What product item type is desired?
* What is the area of interest (geographic region)?
* What percentage of pixels need to be usable?
* etc.

While time range is likely pretty trivial to determine, product item, area of interest, and usable pixels may take a little bit of work. Let's dive into each further

#### Product Item Type

The [product item type](https://developers.planet.com/docs/data/items-assets/) refers to the sensor source (aka satellite type) and basic processing desired. This decision is highly dependent on application, as coverage, revisit rate, spectral bands, and resolution differ between products.

#### Area of Interest

The area of interest is the geographic region for the analysis, given as GeoJSON. If you are familiar with JSON, the format of GeoJSON will likely be easy to grasp. According to <geojson.org>, "GeoJSON is a format for encoding a variety of geographic data structures." The specific geographic data structure we are interested is a `Polygon`. The [GeoJSON wikipedia page](https://en.wikipedia.org/wiki/GeoJSON) gives some great examples of the data structures for the various GeoJSON data structures. 

Some care needs to be given to describing the [position](https://tools.ietf.org/html/rfc7946#section-3.1.1) for each coordinate in a GeoJSON geometry. Each position is specified as `(longitude, latitude)` or `(easting, northing)` (order really matters here!). Also, this may be a surprise, but the same point on earth can have different `(longitude, latitude)` values based on the spatial reference system. Basically, a spatial reference system describes where something is in the real world. But there are thousands of different spatial reference systems. These are separated into two categories: geographic and projected. Geographic coordinate systems model the earth as an ellipsoid (this model is called the `datum`) and describe positions on the surface of the earth (coordinates) in terms of the prime meridian and angle of measure. Projected coordinate systems take this a step further by projecting the geographic coordinates (quite three-dimensional) into two dimensions (the `projection`). Different projections preserve different properties, such as area, angles, or direction for north. There is a rich area of discovery, discussion, and even a little [teasing](https://xkcd.com/977/) (thanks xkcd!) in the world of spatial reference systems.

GeoJSON only supports one spatial reference system, [WGS84](https://spatialreference.org/ref/epsg/wgs-84/). This is a geographic coordinate system describing locations in latitude, longitude. However, many web mapping applications use the Web Mercator projected coordinate system to to describe locations. Confusingly, Web Mercator also describes locations in latitude, longitude. But a `(longitude, latitude)` GeoJSON position given in Web Mercator will **not** end up where you expect if it is not first projected into WGS84. The easiest way to define an aoi that is described in WGS84 is to draw it in [Planet Explorer](https://www.planet.com/explorer) and click the little button "Download AOI as GeoJSON."

#### Usable Pixels

Taking pictures of the earth from space ain't easy. There are a lot of steps and a lot of things have to go right to get a clear shot. Therefore, not every pixel in every image taken from space is useful for analysis. For one, images taken from space are projected into a spatial reference system (discussed briefly above), a process that introduces some NoData (aka outside of the image footprint) pixels into the resulting image. Additionally, clouds cover a great deal of the earth and create cloudy pixels when imaged. While some applications can use cloudy pixels, others cannot. Therefore, the type of pixels that are determined to be 'usable' are often application-specific. To support definition of usable pixels, and filtering based on that definition, Planet provides Usable Data Masks along with Usable Data entries in the imagery metadata. For more details on the Usable Data Mask, check out (Clear for Analysis with Planet’s New Usable Data Masks)[https://www.planet.com/pulse/planets-new-usable-data-masks/]. For more information on the Usable Data metadata entries, see (Usable Data in Planet imagery)[https://developers.planet.com/planetschool/usable-data-in-planet-imagery/].

### Orders API

The core decision around using the orders api is which [product bundle](https://developers.planet.com/docs/orders/product-bundles-reference/) to use. This is the starting point for all processing and there are a lot of options. Once the product is determined, the processing steps (aka tools and toolchains) are defined. Finally, the logistics of the delivery of the imagery are ironed out.

#### Product Bundle

To enable time-series analysis, ARD imagery must be processed so that imagery is consistant across days. This means correcting for differences in camera sensitivities, the relative location of the sun, and the atmospheric conditions. The Analytic Radiance (`analytic`) product bundle provides imagery corrected for difference in camera sensitivities and location of the sun (as radiance) and can also remove the effect of the sun's spectrum by applying the `reflectance` (**TODO** check on and update this name) coefficient provided in the imagery metadata. However, the Analytic Surface Reflectance (`analytic_sr`) product bundle also removes the effect of atmospheric conditions. Therefore, the Analytic Surface Reflectance is the ideal product for ARD.

#### Tools and Toolchains

The [Tools and Toolchains](https://developers.planet.com/docs/orders/tools-toolchains/) functionality in the Orders API are the key to seamlessly creating ARD. Through the API, one can define the pre-processing steps for the data before it is delivered. Given proper definition of the tools and toolchains, data that is delivered is analysis-ready.

#### Delivery

There are a few options for delivery that cater to different use cases. Imagery can be downloaded directly or delivered to [cloud storage](https://developers.planet.com/docs/orders/ordering-delivery/#delivery-to-cloud-storage). When imagery is downloaded, the user can poll for when the order is ready or notifications ([e-mail](https://developers.planet.com/docs/orders/ordering-delivery/#email-notification) or [webhooks](https://developers.planet.com/docs/orders/ordering-delivery/#using-webhooks)) can be used. Additionally, the imagery can be delivered as a [zip archive](https://developers.planet.com/docs/orders/ordering-delivery/#zipping-results).